In [31]:
import boto3
import  sagemaker
from sagemaker import get_execution_role

# set custom bucket
bucket = 'insurance-hope-prediction-v3'
prefix = 'insurance-data-hope'
role   = get_execution_role()

# local fine upload
local_file = 'insurance_pre.csv'

# upload file to your specific bucket
session = sagemaker.Session()
s3_path = session.upload_data(path=local_file, bucket=bucket, key_prefix=prefix)

from utils import append_to_file
append_to_file("output.txt", "s3-path", s3_path)
print(f" File uploaded to your bucket at: {s3_path}")

 File uploaded to your bucket at: s3://insurance-hope-prediction-v3/insurance-data-hope/insurance_pre.csv


In [32]:
import sagemaker

print(sagemaker.get_execution_role())


arn:aws:iam::257034520069:role/service-role/AmazonSageMaker-ExecutionRole-20250828T171031


In [ ]:
"""
# For s3 bucket permission rule add  in your bucket
{
  "Version": "2012-10-17",
  "Statement": [
    {
      "Sid": "AllowSageMakerRoleAccess",
      "Effect": "Allow",
      "Principal": {
        "AWS": "arn:aws:iam::257034520069:role/service-role/AmazonSageMaker-ExecutionRole-20250828T171031"
      },
      "Action": [
        "s3:PutObject",
        "s3:GetObject",
        "s3:ListBucket"
      ],
      "Resource": [
        "arn:aws:s3:::insurance-hope-prediction-v3",
        "arn:aws:s3:::insurance-hope-prediction-v3/*"
      ]
    }
  ]
}

"""

"""
Option - 2 For create policy in IAM Role
    {
    	"Version": "2012-10-17",
    	"Statement": [
    		{
    			"Effect": "Allow",
    			"Action": [
    				"s3:PutObject",
    				"s3:GetObject",
    				"s3:ListObject"
    			],
    			"Resource": [
    				"arn:aws:s3:::insurance-hope-prediction-v3",
    				"arn:aws:s3:::insurance-hope-prediction-v3/*"
    			]
    		}
    	]
    }

"""


In [33]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker import get_execution_role

from utils import append_to_file
session = sagemaker.Session()
role = get_execution_role()
bucket = 'insurance-hope-prediction-v3'
prefix = 'insurance-data-hope'

sklearn_estimator = SKLearn(
    entry_point="train.py",
    role = role,
    instance_type = "ml.m5.large",
    framework_version="1.2-1",
    py_version="py3",
    output_path = f"s3://{bucket}/{prefix}/output",
    sagemaker_session=session
)
# append_to_file("output.txt", "framework_version 1.2-1")

# start training
sklearn_estimator.fit({"train": f"s3://insurance-hope-prediction-v3/insurance-data-hope/"})

INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: sagemaker-scikit-learn-2025-08-31-13-40-22-984


2025-08-31 13:40:26 Starting - Starting the training job...
2025-08-31 13:40:41 Starting - Preparing the instances for training...
2025-08-31 13:40:59 Downloading - Downloading input data...
2025-08-31 13:41:29 Downloading - Downloading the training image...
2025-08-31 13:42:25 Training - Training image download completed. Training in progress../miniconda3/lib/python3.9/site-packages/sagemaker_containers/_server.py:22: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
2025-08-31 13:42:30,042 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-08-31 13:42:30,046 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-08-31 13:42:30,048 sagemaker-training-toolkit INFO     No Neurons detected (normal if no n

In [38]:
image_uri = sklearn_estimator.training_image_uri()
print(image_uri)

model_s3_uri = sklearn_estimator.model_data
print(model_s3_uri)

662702820516.dkr.ecr.eu-north-1.amazonaws.com/sagemaker-scikit-learn:1.2-1-cpu-py3
s3://insurance-hope-prediction-v3/insurance-data-hope/output/sagemaker-scikit-learn-2025-08-31-13-40-22-984/output/model.tar.gz


In [39]:
from  sagemaker.sklearn.model import  SKLearnModel
model = SKLearnModel(
    model_data = model_s3_uri,
    role = role,
    entry_point="inference.py",
    framework_version="1.2-1",
    py_version="py3",
    sagemaker_session=session
)
print("model block loaded")

model block loaded


In [49]:
predictor = model.deploy(
    instance_type='ml.m5.large',
    initial_instance_count=1,
    endpoint_name='insurance-charge-endpoint-v4'  # change version
)


INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2025-08-31-14-26-59-169
INFO:sagemaker:Creating endpoint-config with name insurance-charge-endpoint-v4
INFO:sagemaker:Creating endpoint with name insurance-charge-endpoint-v4


------!

In [55]:
import json
from sagemaker.serializers import JSONSerializer
from sagemaker.deserializers import JSONDeserializer

def predict_insurance_charge(predictor, age, bmi, children, sex_male, smoker_yes):
    predictor.serializer = JSONSerializer()
    predictor.deserializer = JSONDeserializer()
    # Send as 2D array - matches the column order from training
    input_data = [[age, bmi, children, sex_male, smoker_yes]]
    prediction = predictor.predict(input_data)
    return prediction
    
print("Function successfully added")    

Function successfully added


In [ ]:
# from delete_endpoint import delete_endpoints
# aa = delete_endpoints()
# print(aa)

In [56]:
result = predict_insurance_charge(predictor, age=35, bmi=28.4, children=2,
                                  sex_male=1,
                                  smoker_yes=0)
print("Predicted charge:", result)